In [298]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
import os
from datetime import date
import random 

# To plot pretty figures
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

import sklearn
import xgboost as xgb
import lightgbm as lgb
from xgboost.sklearn import XGBClassifier 
from lightgbm import LGBMClassifier
from sklearn import metrics  
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.model_selection import train_test_split  
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import Imputer
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [299]:
pd.set_option('max_rows', 500)
pd.set_option('max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('expand_frame_repr', True)

In [300]:
g_ignore_features=['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV', 'index']

In [301]:
def load_kaggle_csv(filepath, num_rows, root="D:/wangh/Kaggle/HomeCredit/Data"):
    csv_path = os.path.join(root, filepath)
    
    if not os.path.isfile(csv_path) :
            csv_path = os.path.join("E:/SparkExerciseData/Kaggle", filepath)
        
    return pd.read_csv(csv_path, nrows = num_rows)

In [302]:
@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

In [303]:
# One-hot encoding for categorical columns with get_dummies
def one_hot_encoder(df, nan_as_category = True):
    original_columns = list(df.columns)
    categorical_columns = [col for col in df.columns if df[col].dtype == 'object']
    df = pd.get_dummies(df, columns= categorical_columns, dummy_na= nan_as_category)
    new_columns = [c for c in df.columns if c not in original_columns]
    return df, new_columns

In [304]:
# Preprocess application_train.csv and application_test.csv
def application_train_test(num_rows = None, nan_as_category = True):
    # Read data and merge
    df = load_kaggle_csv('application_train.csv', num_rows)
    test_df = load_kaggle_csv('application_test.csv', num_rows)
    print("Train samples: {}, test samples: {}".format(len(df), len(test_df)))
    df = df.append(test_df).reset_index()
    
    # Categorical features: Binary features and One-Hot encoding
    for bin_feature in ['CODE_GENDER', 'FLAG_OWN_CAR', 'FLAG_OWN_REALTY']:
        df[bin_feature], uniques = pd.factorize(df[bin_feature])
    df, cat_cols = one_hot_encoder(df, nan_as_category)
    # NaN values for DAYS_EMPLOYED: 365.243 -> nan
    df['DAYS_EMPLOYED'].replace(365243, np.nan, inplace= True)
    
    #AMT_INCOME_TOTAL 117000000 -> nan
    df['AMT_INCOME_TOTAL'].replace(117000000, np.nan, inplace= True)
    
    # Some simple new features (percentages)
    df['DAYS_EMPLOYED_PERC'] = df['DAYS_EMPLOYED'] / df['DAYS_BIRTH']
    df['INCOME_CREDIT_PERC'] = df['AMT_INCOME_TOTAL'] / df['AMT_CREDIT']
    df['INCOME_PER_PERSON'] = df['AMT_INCOME_TOTAL'] / df['CNT_FAM_MEMBERS']
    df['ANNUITY_INCOME_PERC'] = df['AMT_ANNUITY'] / df['AMT_INCOME_TOTAL']
    del test_df
    gc.collect()
    return df

In [305]:
# Preprocess bureau.csv and bureau_balance.csv
def bureau_and_balance(num_rows = None, nan_as_category = True):
    bureau = load_kaggle_csv('bureau.csv', num_rows)
    bb = load_kaggle_csv('bureau_balance.csv', num_rows)
    bb, bb_cat = one_hot_encoder(bb, nan_as_category)
    bureau, bureau_cat = one_hot_encoder(bureau, nan_as_category)
    
    # Bureau balance: Perform aggregations and merge with bureau.csv
    bb_aggregations = {'MONTHS_BALANCE': ['min', 'max', 'size']}
    for col in bb_cat:
        bb_aggregations[col] = ['mean']
    bb_agg = bb.groupby('SK_ID_BUREAU').agg(bb_aggregations)
    bb_agg.columns = pd.Index([e[0] + "_" + e[1].upper() for e in bb_agg.columns.tolist()])
    bureau = bureau.join(bb_agg, how='left', on='SK_ID_BUREAU')
    bureau.drop(columns= 'SK_ID_BUREAU', inplace= True)
    del bb, bb_agg
    gc.collect()
    
    # Bureau and bureau_balance numeric features
    num_aggregations = {
        'DAYS_CREDIT': ['min', 'max', 'mean', 'var'],
        'CREDIT_DAY_OVERDUE': ['max', 'mean'],
        'DAYS_CREDIT_ENDDATE': ['min', 'max', 'mean'],
        'AMT_CREDIT_MAX_OVERDUE': ['mean'],
        'CNT_CREDIT_PROLONG': ['sum'],
        'AMT_CREDIT_SUM': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_DEBT': ['max', 'mean', 'sum'],
        'AMT_CREDIT_SUM_OVERDUE': ['mean'],
        'AMT_CREDIT_SUM_LIMIT': ['mean', 'sum'],
        'DAYS_CREDIT_UPDATE': ['min', 'max', 'mean'],
        'AMT_ANNUITY': ['max', 'mean'],
        'MONTHS_BALANCE_MIN': ['min'],
        'MONTHS_BALANCE_MAX': ['max'],
        'MONTHS_BALANCE_SIZE': ['mean', 'sum']
    }
    # Bureau and bureau_balance categorical features
    cat_aggregations = {}
    for cat in bureau_cat: cat_aggregations[cat] = ['mean']
    for cat in bb_cat: cat_aggregations[cat + "_MEAN"] = ['mean']
    
    bureau_agg = bureau.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    bureau_agg.columns = pd.Index(['BURO_' + e[0] + "_" + e[1].upper() for e in bureau_agg.columns.tolist()])
    # Bureau: Active credits - using only numerical aggregations
    active = bureau[bureau['CREDIT_ACTIVE_Active'] == 1]
    active_agg = active.groupby('SK_ID_CURR').agg(num_aggregations)
    active_agg.columns = pd.Index(['ACT_' + e[0] + "_" + e[1].upper() for e in active_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(active_agg, how='left', on='SK_ID_CURR')
    del active, active_agg
    gc.collect()
    # Bureau: Closed credits - using only numerical aggregations
    closed = bureau[bureau['CREDIT_ACTIVE_Closed'] == 1]
    closed_agg = closed.groupby('SK_ID_CURR').agg(num_aggregations)
    closed_agg.columns = pd.Index(['CLS_' + e[0] + "_" + e[1].upper() for e in closed_agg.columns.tolist()])
    bureau_agg = bureau_agg.join(closed_agg, how='left', on='SK_ID_CURR')
    del closed, closed_agg, bureau
    gc.collect()
    return bureau_agg

In [306]:
# Preprocess previous_applications.csv
def previous_applications(num_rows = None, nan_as_category = True):
    prev = load_kaggle_csv('previous_application.csv', num_rows)
    prev, cat_cols = one_hot_encoder(prev, nan_as_category= True)
    # Days 365.243 values -> nan
    prev['DAYS_FIRST_DRAWING'].replace(365243, np.nan, inplace= True)
    prev['DAYS_FIRST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE_1ST_VERSION'].replace(365243, np.nan, inplace= True)
    prev['DAYS_LAST_DUE'].replace(365243, np.nan, inplace= True)
    prev['DAYS_TERMINATION'].replace(365243, np.nan, inplace= True)
    # Add feature: value ask / value received percentage
    prev['APP_CREDIT_PERC'] = prev['AMT_APPLICATION'] / prev['AMT_CREDIT']
    # Previous applications numeric features
    num_aggregations = {
        'AMT_ANNUITY': ['min', 'max', 'mean'],
        'AMT_APPLICATION': ['min', 'max', 'mean'],
        'AMT_CREDIT': ['min', 'max', 'mean'],
        'APP_CREDIT_PERC': ['min', 'max', 'mean', 'var'],
        'AMT_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'AMT_GOODS_PRICE': ['min', 'max', 'mean'],
        'HOUR_APPR_PROCESS_START': ['min', 'max', 'mean'],
        'RATE_DOWN_PAYMENT': ['min', 'max', 'mean'],
        'DAYS_DECISION': ['min', 'max', 'mean'],
        'CNT_PAYMENT': ['mean', 'sum'],
    }
    # Previous applications categorical features
    cat_aggregations = {}
    for cat in cat_cols:
        cat_aggregations[cat] = ['mean']
    
    prev_agg = prev.groupby('SK_ID_CURR').agg({**num_aggregations, **cat_aggregations})
    prev_agg.columns = pd.Index(['PREV_' + e[0] + "_" + e[1].upper() for e in prev_agg.columns.tolist()])
    # Previous Applications: Approved Applications - only numerical features
    approved = prev[prev['NAME_CONTRACT_STATUS_Approved'] == 1]
    approved_agg = approved.groupby('SK_ID_CURR').agg(num_aggregations)
    approved_agg.columns = pd.Index(['APR_' + e[0] + "_" + e[1].upper() for e in approved_agg.columns.tolist()])
    prev_agg = prev_agg.join(approved_agg, how='left', on='SK_ID_CURR')
    # Previous Applications: Refused Applications - only numerical features
    refused = prev[prev['NAME_CONTRACT_STATUS_Refused'] == 1]
    refused_agg = refused.groupby('SK_ID_CURR').agg(num_aggregations)
    refused_agg.columns = pd.Index(['REF_' + e[0] + "_" + e[1].upper() for e in refused_agg.columns.tolist()])
    prev_agg = prev_agg.join(refused_agg, how='left', on='SK_ID_CURR')
    del refused, refused_agg, approved, approved_agg, prev
    gc.collect()
    return prev_agg

In [307]:
# Preprocess POS_CASH_balance.csv
def pos_cash(num_rows = None, nan_as_category = True):
    pos = load_kaggle_csv('POS_CASH_balance.csv', num_rows)
    pos, cat_cols = one_hot_encoder(pos, nan_as_category= True)
    # Features
    aggregations = {
        'MONTHS_BALANCE': ['max', 'mean', 'size'],
        'SK_DPD': ['max', 'mean'],
        'SK_DPD_DEF': ['max', 'mean']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    
    pos_agg = pos.groupby('SK_ID_CURR').agg(aggregations)
    pos_agg.columns = pd.Index(['POS_' + e[0] + "_" + e[1].upper() for e in pos_agg.columns.tolist()])
    # Count pos cash accounts
    pos_agg['POS_COUNT'] = pos.groupby('SK_ID_CURR').size()
    del pos
    gc.collect()
    return pos_agg

In [308]:
# Preprocess installments_payments.csv
def installments_payments(num_rows = None, nan_as_category = True):
    ins = load_kaggle_csv('installments_payments.csv', num_rows)
    ins, cat_cols = one_hot_encoder(ins, nan_as_category= True)
    # Percentage and difference paid in each installment (amount paid and installment value)
    ins['PAYMENT_PERC'] = ins['AMT_PAYMENT'] / ins['AMT_INSTALMENT']
    ins['PAYMENT_DIFF'] = ins['AMT_INSTALMENT'] - ins['AMT_PAYMENT']
    # Days past due and days before due (no negative values)
    ins['DPD'] = ins['DAYS_ENTRY_PAYMENT'] - ins['DAYS_INSTALMENT']
    ins['DBD'] = ins['DAYS_INSTALMENT'] - ins['DAYS_ENTRY_PAYMENT']
    ins['DPD'] = ins['DPD'].apply(lambda x: x if x > 0 else 0)
    ins['DBD'] = ins['DBD'].apply(lambda x: x if x > 0 else 0)
    # Features: Perform aggregations
    aggregations = {
        'NUM_INSTALMENT_VERSION': ['nunique'],
        'DPD': ['max', 'mean', 'sum'],
        'DBD': ['max', 'mean', 'sum'],
        'PAYMENT_PERC': ['max', 'mean', 'sum', 'var'],
        'PAYMENT_DIFF': ['max', 'mean', 'sum', 'var'],
        'AMT_INSTALMENT': ['max', 'mean', 'sum'],
        'AMT_PAYMENT': ['min', 'max', 'mean', 'sum'],
        'DAYS_ENTRY_PAYMENT': ['max', 'mean', 'sum']
    }
    for cat in cat_cols:
        aggregations[cat] = ['mean']
    ins_agg = ins.groupby('SK_ID_CURR').agg(aggregations)
    ins_agg.columns = pd.Index(['INS_' + e[0] + "_" + e[1].upper() for e in ins_agg.columns.tolist()])
    # Count installments accounts
    ins_agg['INS_COUNT'] = ins.groupby('SK_ID_CURR').size()
    del ins
    gc.collect()
    return ins_agg

In [309]:
# Preprocess credit_card_balance.csv
def credit_card_balance(num_rows = None, nan_as_category = True):
    cc = load_kaggle_csv('credit_card_balance.csv', num_rows)
    cc, cat_cols = one_hot_encoder(cc, nan_as_category= True)
    # General aggregations
    cc.drop(columns = ['SK_ID_PREV'], inplace = True)
    cc_agg = cc.groupby('SK_ID_CURR').agg(['min', 'max', 'mean', 'sum', 'var'])
    cc_agg.columns = pd.Index(['CC_' + e[0] + "_" + e[1].upper() for e in cc_agg.columns.tolist()])
    # Count credit card lines
    cc_agg['CC_COUNT'] = cc.groupby('SK_ID_CURR').size()
    del cc
    gc.collect()
    return cc_agg

In [310]:
def print_AUC(alg, input_train, input_Y, input_test, input_test_Y) :
    predictions_train = alg.predict(input_train)
    predictions_prob_train = alg.predict_proba(input_train)[:,1]
    print("Train Accuracy Score:", metrics.accuracy_score(input_Y, predictions_train))
    print("Train AUC Score:", metrics.roc_auc_score(input_Y, predictions_prob_train))

    predictions = alg.predict(input_test)
    predictions_prob = alg.predict_proba(input_test)[:,1]

    print("Accuracy Score:", metrics.accuracy_score(input_test_Y, predictions))
    print("AUC Score:", metrics.roc_auc_score(input_test_Y, predictions_prob))

In [311]:
def cv_alg(alg, cv_df, num_folds=5, stratified = True):
    print("Starting CV {}. CV shape: {}".format(alg.__class__.__name__,cv_df.shape))
    
    global g_ignore_features
    
    if stratified:
        folds = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits=num_folds, shuffle=True, random_state=1001)
        
    feats = [f for f in cv_df.columns if f not in g_ignore_features]
    
    cv_df_feats = cv_df[feats]
    cv_df_TARGET = cv_df['TARGET']
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(cv_df_feats, cv_df_TARGET)):
        train_x, train_y = cv_df_feats.iloc[train_idx], cv_df_TARGET.iloc[train_idx]
        valid_x, valid_y = cv_df_feats.iloc[valid_idx], cv_df_TARGET.iloc[valid_idx]   
        alg.fit(train_x, train_y)
        print_AUC(alg, train_x, train_y, valid_x,  valid_y)


In [312]:
from sklearn.model_selection import StratifiedShuffleSplit

def fit_alg(alg, fit_train_df, fit_val_df=None):
    print("Starting fit {}. fit shape: {}".format(alg.__class__.__name__,fit_train_df.shape))

    global g_ignore_features
    
    feats = [f for f in fit_train_df.columns if f not in g_ignore_features]
    
    fit_X_train = fit_train_df[feats]
    fit_y_train = fit_train_df['TARGET']
    
    if fit_val_df is None :
        alg.fit(fit_X_train, fit_y_train)
    else :
        fit_X_test = fit_val_df[feats]
        fit_y_test = fit_val_df['TARGET']          
        alg.fit(fit_X_train, fit_y_train)

        print_AUC(alg, fit_X_train, fit_y_train, fit_X_test,  fit_y_test)

In [313]:
# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, clf, num_folds, stratified = False):
    global g_ignore_features
    
    # Divide in training/validation and test data
    train_df = df[df['TARGET'].notnull()]
    test_df = df[df['TARGET'].isnull()]
    print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))

    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=1001)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=1001)
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    feature_importance_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in g_ignore_features]
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df['TARGET'])):
        train_x, train_y = train_df[feats].iloc[train_idx], train_df['TARGET'].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df['TARGET'].iloc[valid_idx]

        # LightGBM parameters found by Bayesian optimization
       # clf = LGBMClassifier(
       #     nthread=4,
       #     n_estimators=10000,
       #     learning_rate=0.02,
       #     num_leaves=34,
       #     colsample_bytree=0.9497036,
       #     subsample=0.8715623,
       #     max_depth=8,
       #     reg_alpha=0.041545473,
       #     reg_lambda=0.0735294,
       #     min_split_gain=0.0222415,
       #     min_child_weight=39.3259775,
       #    silent=-1,
       #    verbose=-1, )

        clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
            eval_metric= 'auc', verbose= 100, early_stopping_rounds= 100)

        oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
        sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        print('Fold %2d AUC : %.6f' % (n_fold + 1, roc_auc_score(valid_y, oof_preds[valid_idx])))
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()

    print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    # Write submission file and plot feature importance
    test_df['TARGET'] = sub_preds
    test_df[['SK_ID_CURR', 'TARGET']].to_csv(submission_file_name, index= False)
    display_importances(feature_importance_df)
    return feature_importance_df

In [314]:
# Display/plot feature importance
def display_importances(feature_importance_df_):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig('lgbm_importances-01.png')

# Data preparation

In [315]:
debug = False

In [ ]:
#def prepare_data(debug = False):
num_rows = 10000 if debug else None
df = application_train_test(num_rows)
with timer("Process bureau and bureau_balance"):
    bureau = bureau_and_balance(num_rows)
    print("Bureau df shape:", bureau.shape)
    df = df.join(bureau, how='left', on='SK_ID_CURR')

with timer("Process previous_applications"):
    prev = previous_applications(num_rows)
    print("Previous applications df shape:", prev.shape)
    df = df.join(prev, how='left', on='SK_ID_CURR')

with timer("Process POS-CASH balance"):
    pos = pos_cash(num_rows)
    print("Pos-cash balance df shape:", pos.shape)
    df = df.join(pos, how='left', on='SK_ID_CURR')

with timer("Process installments payments"):
    ins = installments_payments(num_rows)
    print("Installments payments df shape:", ins.shape)
    df = df.join(ins, how='left', on='SK_ID_CURR')

with timer("Process credit card balance"):
    cc = credit_card_balance(num_rows)
    print("Credit card balance df shape:", cc.shape)
    df = df.join(cc, how='left', on='SK_ID_CURR')
    
del bureau
del prev
del pos
del ins  
del cc
gc.collect()
#with timer("Run LightGBM with kfold"):
    #feat_importance = kfold_lightgbm(df, num_folds= 5, stratified = False)



Train samples: 307511, test samples: 48744
Bureau df shape: (305811, 122)
Process bureau and bureau_balance - done in 26s


In [ ]:
train_df = df[df['TARGET'].notnull()]
test_df = df[df['TARGET'].isnull()]

In [ ]:
import math
ignore_features_median =  ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']
relevant_features_median = [col for col in df.columns if col not in ignore_features_median]
df_imp = df.replace([np.inf, -np.inf], np.nan)

for col in relevant_features_median :
    col_median = df_imp[col].median()
    if math.isnan(col_median) : 
        print("is nan:", col)
        col_median=0.0
    df_imp[col].fillna(col_median, inplace=True)
train_df_imp = df_imp[df_imp['TARGET'].notnull()]
test_df_imp = df_imp[df_imp['TARGET'].isnull()]

In [ ]:
df_imp.columns[np.isfinite(df_imp).all()==False]

In [ ]:
split = StratifiedShuffleSplit(n_splits=1, test_size= 0.3, random_state=1001)
for train_index, val_index in split.split(train_df, train_df['TARGET']):
    X_train, X_val = train_df.loc[train_index], train_df.loc[val_index]
    X_train_imp, X_val_imp = train_df_imp.loc[train_index], train_df_imp.loc[val_index]

In [ ]:
train_df['TARGET'].head()

In [ ]:
X_train['TARGET'].head()

In [ ]:
df.shape, X_train.shape, X_val.shape, X_train_imp.shape, X_val_imp.shape

In [ ]:
np.isfinite(X_train_imp).all()

In [ ]:
X_train_imp.columns[np.isfinite(X_train_imp).all()==False]

In [ ]:
train_df[relevant_features_median].iloc[5687:5689, 654:657].head()

In [ ]:
train_df_imp[relevant_features_median].iloc[5687:5689, 654:657].head()

In [ ]:
train_df_imp.head()

In [ ]:
gc.collect()

# Final Models

In [ ]:
lgb_clf = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=30, max_depth=8,
    learning_rate=0.01, n_estimators=5000,  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=27, n_jobs=-1, silent=False
)

with timer("fit lgb.LGBMClassifier"):
    fit_alg(lgb_clf, X_train, X_val)

In [ ]:
xgb_clf = XGBClassifier(
        learning_rate =0.05,
        n_estimators=2000,
        max_depth=5,
        min_child_weight=5,
        gamma=0.1,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_alpha=0.1,
        objective= 'binary:logistic',
        nthread=8,
        silent=False,
        scale_pos_weight=1,
        seed=27)

with timer("fit XGBClassifier"):
    fit_alg(xgb_clf, X_train, X_val)
    

In [ ]:
log_clf = LogisticRegression(penalty='l1', C=1, random_state=1001)

with timer("fit LogisticRegression"):
    fit_alg(log_clf, X_train_imp, X_val_imp)

In [ ]:
rnd_clf = RandomForestClassifier(n_estimators=5000, 
                        max_leaf_nodes=30, 
                        max_features=0.1,
                        max_depth=5,
                        bootstrap=True, n_jobs=-1, random_state=27)
with timer("fit RandomForestClassifier"):
    fit_alg(rnd_clf, X_train_imp, X_val_imp)

In [ ]:
gc.collect()

In [ ]:
X_val['TARGET'].head(20)

In [ ]:
X_val_imp_feats.head()

In [ ]:
pd.concat([X_val['TARGET'].reset_index(),pd.DataFrame(seclvl_train_all)], axis=1)

In [ ]:
xgb_clf.predict_proba(X_val_feats)[:,1]

In [ ]:
pd.DataFrame(seclvl_train_all).head()

In [ ]:
level2_df

In [ ]:
feats = [f for f in test_df.columns if f not in g_ignore_features]
X_val_imp_feats = X_val_imp[feats]
X_val_feats = X_val[feats]

gbm_level2 = xgb.XGBClassifier(
 learning_rate = 0.01,
 n_estimators= 1000,
 max_depth=2,
 min_child_weight=1,
 gamma=0.1,  
 subsample=0.6, 
 colsample_bytree=1,
 reg_alpha = 0.2,
 objective= 'binary:logistic',
 nthread= -1,
 silent=False,
 scale_pos_weight=1)

seclvl_train_all = np.concatenate(( np.array([log_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([rnd_clf.predict_proba(X_val_imp_feats)[:,1]]).T,
                                np.array([xgb_clf.predict_proba(X_val_feats)[:,1]]).T,
                                np.array([lgb_clf.predict_proba(X_val_feats)[:,1]]).T
                            ), axis=1)

level2_df = pd.concat([X_val['TARGET'].reset_index(),pd.DataFrame(seclvl_train_all)], axis=1)
cv_alg(gbm_level2, level2_df)
fit_alg(gbm_level2, level2_df)

# Generate the final result

In [ ]:
submission_path= "D:/wangh/Kaggle/HomeCredit/result_submission7_{}{}.csv"

feats = [f for f in test_df.columns if f not in g_ignore_features]

for clf, test_data_df in [
        (lgb_clf, test_df), 
        (xgb_clf, test_df),
        (log_clf, test_df_imp), 
        (rnd_clf, test_df_imp),
    ]: 
    result_df=pd.DataFrame()
    result_df['SK_ID_CURR'] = test_df['SK_ID_CURR']
    result_df['TARGET'] = clf.predict_proba(test_data_df[feats])[:, 1]

    submission_filepath = submission_path.format(clf.__class__.__name__,'_1stlvl')
    result_df.to_csv(submission_filepath, index=False, float_format='%.6f')
    
    del result_df
    gc.collect()


In [ ]:
seclvl_test=test_df[feats]

feats = [f for f in test_df.columns if f not in g_ignore_features]
test_df_imp_feats = test_df_imp[feats]
test_df_feats = test_df[feats]


seclvl_train_all = np.concatenate(( np.array([log_clf.predict_proba(test_df_imp_feats)[:,1]]).T,
                                np.array([rnd_clf.predict_proba(test_df_imp_feats)[:,1]]).T,
                                np.array([xgb_clf.predict_proba(test_df_feats)[:,1]]).T,
                                np.array([lgb_clf.predict_proba(test_df_feats)[:,1]]).T
                            ), axis=1)

result_df=pd.DataFrame()
result_df['SK_ID_CURR'] = test_df['SK_ID_CURR']
result_df['TARGET'] = gbm_level2.predict_proba(pd.DataFrame(seclvl_train_all))[:, 1]

submission_filePath = submission_path.format(gbm_level2.__class__.__name__, '_2ndlvl')
result_df.to_csv(submission_filePath, index=False, float_format='%.6f')

del test_df_imp_feats
del test_df_feats
del result_df
gc.collect()


# Tune Models

In [32]:
feats = [f for f in train_df.columns if f not in ['TARGET','SK_ID_CURR','SK_ID_BUREAU','SK_ID_PREV']]

df_feats = train_df[feats]
df_TARGET = train_df['TARGET']

In [ ]:
lgb_train = lgb.Dataset(data=df_feats, label=df_TARGET)
params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'auc', 
          'learning_rate': 0.01, 'num_leaves': 48, 'num_iteration': 5000, 'verbose': 10 ,
          'colsample_bytree':.8, 'subsample':.9, 'max_depth':7, 'reg_alpha':.1, 'reg_lambda':.1, 
          'min_split_gain':.01, 'min_child_weight':1}
lgb_result=lgb.cv(params, lgb_train, stratified=True, metrics={'auc'}, num_boost_round=5000,
                      nfold=5, early_stopping_rounds=150, verbose_eval=20)
lgb_result

C:\ProgramData\Anaconda3\lib\site-packages\lightgbm\engine.py:390: UserWarning: Found `num_iteration` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[20]	cv_agg's auc: 0.738788 + 0.0032822
[40]	cv_agg's auc: 0.739973 + 0.00340554


In [ ]:
#grid search
lgb_param_test1 = {
    'max_depth':[8], #range(2,10,2),
    'num_leaves':[30], #range(30,60,10),
    'min_child_samples':range(200,1000,300)
    'min_child_weigh':range(20,100,20)
    
}
lgb_gsearch1 = GridSearchCV(estimator = lgb.LGBMClassifier(
    boosting_type='gbdt', num_leaves=48, max_depth=7,
    learning_rate=0.01, n_estimators=len(lgb_result['auc-mean']),  
    objective='binary', min_split_gain=0.02, min_child_weight=1, 
    min_child_samples=300, subsample=0.85, subsample_freq=0, colsample_bytree=0.95,
    reg_alpha=0.1, reg_lambda=0.1, random_state=1001, n_jobs=-1, silent=False
), param_grid = lgb_param_test1, scoring='roc_auc',n_jobs=4,verbose=10,iid=False, cv=5)


lgb_gsearch1.fit(df_feats, df_TARGET, eval_metric='auc', verbose = 50)
lgb_gsearch1.grid_scores_, lgb_gsearch1.best_params_, lgb_gsearch1.best_score_ 

In [ ]:
def modelfit(alg, train, train_target, useTrainCV=True, cv_folds=5, early_stopping_rounds=150, val_percent=0.2):
    
    split = StratifiedShuffleSplit(n_splits=1, test_size= val_percent, random_state=1001)
    for train_index, test_index in split.split(train, train_target):
        dtrain, dtest = train.loc[train_index], train.loc[test_index]
        dtrain_target, dtest_target = train_target.loc[train_index], train_target.loc[test_index]
            
            
    if useTrainCV:
        xgb_param = alg.get_xgb_params()
        xgtrain = xgb.DMatrix(dtrain.values, label=dtrain_target.values)
        cvresult = xgb.cv(xgb_param, xgtrain, num_boost_round=alg.get_params()['n_estimators'], nfold=cv_folds,
            metrics={'auc'}, early_stopping_rounds=early_stopping_rounds)
        print("n_estimators use : %d" % cvresult.shape[0])
        alg.set_params(n_estimators=cvresult.shape[0])
    
    #Fit the algorithm on the data
    alg.fit(dtrain, dtrain_target,eval_metric='auc')
        
    #Predict training set:
    dtrain_predictions = alg.predict(dtrain)
    dtrain_predprob = alg.predict_proba(dtrain)[:,1]
        
    #Print model report:
    print("\ndtrain Model Report")
    print("dtrain Accuracy : %.4g" % metrics.accuracy_score(dtrain_target, dtrain_predictions))
    print("dtrain AUC Score: %f" % metrics.roc_auc_score(dtrain_target, dtrain_predprob)) 
    
    #     Predict on validation data:
    dtest_predictions = alg.predict(dtest)
    dtest_predprob = alg.predict_proba(dtest)[:,1]
        
    #Print model report:
    print("\ndtest Model Report")
    print("dtest Accuracy : %.4g" % metrics.accuracy_score(dtest_target, dtest_predictions))
    print("dtest AUC Score: %f" % metrics.roc_auc_score(dtest_target, dtest_predprob))

In [ ]:
xgb1 = xgb.XGBClassifier(
        learning_rate=0.1,
        n_estimators=1000,
        max_depth=5,
        min_child_weight=1,
        gamma=0,
        subsample=0.8,
        colsample_bytree=0.8,
        objective= 'binary:logistic',
        nthread=4,
        scale_pos_weight=1,
        silent=False,    
        seed=1001)
modelfit(xgb1, df_feats, df_TARGET)

In [ ]:
param_test1 = {
    'max_depth':range(3,10,2),
    'min_child_weight':range(1,6,2)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, 
                                                  n_estimators=xgb1.get_params()['n_estimators'], 
                                        max_depth=5,
                                        min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1, 
                                                         random_state=1001, silent=False,  seed=1001, verbose=10), 
                       param_grid = param_test1, scoring='roc_auc',n_jobs=4,verbose=10,iid=False, cv=5)
gsearch1.fit(df_feats, df_TARGET)

In [ ]:
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

In [ ]:
param_test3 = {
    'gamma':[i/10.0 for i in range(0,5)]
}
gsearch3 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=xgb1.get_params()['n_estimators'],
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=0, subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=4, scale_pos_weight=1,
                                                  random_state=27, silent=False, seed=27), 
                       param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False,verbose=10, cv=5)
gsearch3.fit(df_feats, df_TARGET)

In [ ]:
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_

In [ ]:
param_test4 = {
    'subsample':[i/10.0 for i in range(6,10)],
    'colsample_bytree':[i/10.0 for i in range(6,10)]
}
gsearch4 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=1000, 
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=gsearch3.best_params_['gamma'], 
                                        subsample=0.8, colsample_bytree=0.8,
                                        objective= 'binary:logistic', nthread=2, verbose=10,silent=False,
                                                  scale_pos_weight=1,seed=27), 
                       param_grid = param_test4, scoring='roc_auc',n_jobs=4,iid=False, verbose=10,cv=5)
gsearch4.fit(df_feats, df_TARGET)

In [ ]:
gsearch4.grid_scores_, gsearch4.best_params_, gsearch4.best_score_

In [ ]:
param_test6 = {
    'reg_alpha':[1e-5, 1e-2, 0.1, 1, 100]
}
gsearch6 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=232,
                                                  max_depth=gsearch1.best_params_['max_depth'],
                                        min_child_weight=gsearch1.best_params_['min_child_weight'],
                                                  gamma=gsearch3.best_params_['gamma'],  
                                                  subsample=gsearch4.best_params_["subsample"], 
                                        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
                                                  silent=False,
                                        objective= 'binary:logistic', nthread=2, verbose=10,
                                                  scale_pos_weight=1,seed=27), 
                       param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(df_feats, df_TARGET)

In [ ]:
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_

In [ ]:
xgb3 = XGBClassifier(
        learning_rate =0.1,
        n_estimators=1000,
        max_depth==gsearch1.best_params_['max_depth'],
        min_child_weight=gsearch1.best_params_['min_child_weight'],
        gamma=gsearch3.best_params_['gamma'], 
        subsample=gsearch4.best_params_["subsample"],
        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
        reg_alpha=gsearch6.best_params_["reg_alpha"],
        objective= 'binary:logistic',
        nthread=8,
        scale_pos_weight=1,
        silent=False,
        seed=27)
modelfit(xgb3, df_feats, df_TARGET)

In [ ]:
xgb3

In [ ]:
xgb4 = XGBClassifier(
        learning_rate =0.01,
        n_estimators=5000,
        max_depth=gsearch1.best_params_['max_depth'],
        min_child_weight=gsearch1.best_params_['min_child_weight'],
        gamma=gsearch3.best_params_['gamma'],
        subsample=gsearch4.best_params_["subsample"],
        colsample_bytree=gsearch4.best_params_["colsample_bytree"],
        reg_alpha=gsearch6.best_params_["reg_alpha"],
        objective= 'binary:logistic',
        nthread=8,
        silent=False,
        scale_pos_weight=1,
        seed=27)
modelfit(xgb4, df_feats, df_TARGET)

In [ ]:
xgb4